In [ ]:
import os
import sys
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import pandas as pd
from joblib import Parallel, delayed
import seaborn as sns


cwd = os.getcwd()
model_path = os.path.abspath(os.path.join(cwd, os.pardir, 'models'))
sys.path.append(model_path)
from cumul_diamond_ODE_model import *
from cumul_treated_diamond_ODE_model import *

In [ ]:
untreated_model = cumulDiamondODEModel()
treated_model = cumulTreatedDiamondODEModel()
times = np.linspace(0,31,100)
n_simulations = 100
nu = 2 / 4.3
inferred_parameters = pd.read_csv('../data/inferred_parameters.csv')
data = pd.read_csv('../data/data_cases_symp.csv')
data['onset_date'] = pd.to_datetime(data['onset_date'], format='%d-%b')

In [ ]:
# Pre-sample parameter sets to ensure both models use the same parameters for each simulation
sampled_indices = np.random.randint(0, inferred_parameters.shape[0], size=n_simulations)
sampled_parameters = [inferred_parameters.iloc[idx].to_list() for idx in sampled_indices]

coverage_array = [(1, 1), (1, 0), (0, 1), (0.5, 0.5)]
efficacy_array = np.linspace(0, 1, 100)

def simulate_models(params, times):
    untreated_values = untreated_model.simulate(params[:8], times)
    cumul_infected_untreated_passengers = untreated_values[-1,22]
    cumul_infected_untreated_crew = untreated_values[-1,23]
    total_untreated_infections = cumul_infected_untreated_passengers + cumul_infected_untreated_crew

    averted_results = {}
    for eff in efficacy_array:
        for coverage in coverage_array:
            treated_values = treated_model.simulate(params[:8] + [eff, eff, coverage[0], coverage[1]], times)
            cumul_infected_treated_passengers = treated_values[-1,22]
            cumul_infected_treated_crew = treated_values[-1,23]
            total_treated_infections = cumul_infected_treated_passengers + cumul_infected_treated_crew

            infections_averted_percentage = 100 * (total_untreated_infections - total_treated_infections) / total_untreated_infections if total_untreated_infections > 0 else 0
            
            averted_results.setdefault((eff, coverage[0], coverage[1]), []).append(infections_averted_percentage)

    return averted_results

# Execute simulations in parallel
all_results = Parallel(n_jobs=-1)(delayed(simulate_models)(params, times) for params in sampled_parameters)

# Combine results from all simulations
combined_results = {}
for result in all_results:
    for key, values in result.items():
        combined_results.setdefault(key, []).extend(values)

# Calculate percentiles for each combination of efficacy and coverage
percentile_results = []
for key, values in combined_results.items():
    efficacy, passenger_coverage, crew_coverage = key
    median_averted = np.percentile(values, 50)
    p5_averted = np.percentile(values, 5)
    p95_averted = np.percentile(values, 95)
    percentile_results.append((efficacy, passenger_coverage, crew_coverage, median_averted, p5_averted, p95_averted))

# Convert results to a DataFrame
columns = ["Efficacy", "Passenger Coverage", "Crew Coverage", "Median Percentage Infections Averted", "5th Percentile Percentage Infections Averted", "95th Percentile Percentage Infections Averted"]
results_df = pd.DataFrame(percentile_results, columns=columns)


In [ ]:
# Setting the color palette to viridis
colors = plt.cm.viridis(np.linspace(0, 1, len(coverage_array)))

# Creating the plot
plt.figure(figsize=(6, 5))
plt.rcParams['font.sans-serif']=='Arial'

for i, coverage in enumerate(coverage_array):
    # Filtering the data for the current coverage scenario
    scenario_data = results_df[(results_df['Passenger Coverage'] == coverage[0]) & (results_df['Crew Coverage'] == coverage[1])]

    # Sorting by efficacy for consistent plotting
    scenario_data = scenario_data.sort_values('Efficacy')

    # Plotting the median percentage of infections averted
    plt.plot(scenario_data['Efficacy'] * 100, scenario_data['Median Percentage Infections Averted'], color=colors[i], label=f'Passenger: {int(coverage[0]*100)}%, Crew: {int(coverage[1]*100)}%')

    # Adding the shaded region for the 5th-95th percentile
    plt.fill_between(scenario_data['Efficacy'] * 100, scenario_data['5th Percentile Percentage Infections Averted'], scenario_data['95th Percentile Percentage Infections Averted'], color=colors[i], alpha=0.3)


plt.plot(np.nan, np.nan, color='black', label='Median')
plt.fill_between([np.nan,np.nan], [np.nan, np.nan], alpha=0.3, color='black', label='90% confidence interval')

# Formatting the y-axis to show percentage
plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter())

# Formatting the x-axis to show percentage
plt.gca().xaxis.set_major_formatter(mtick.PercentFormatter())

# Adjusting y-axis limits
plt.ylim(0, 100)
plt.xlim(0, 100)

# Adding labels and title
plt.xlabel('Efficacy', weight='bold', fontsize=12)
plt.ylabel('Infections averted', weight='bold', fontsize=12)
plt.legend(title='Coverage', loc='lower right', fontsize=8)

# Show the plot
plt.tight_layout()

# Define the directory path and the filename
directory = '../figures'
filename = 'figure3c.svg'

# Check if the directory exists
if not os.path.exists(directory):
    # If the directory does not exist, create it
    os.makedirs(directory)

# Save the figure
plt.savefig(os.path.join(directory, filename), bbox_inches='tight')
plt.show()
